# **REFACTORING_MINER_bug**

This notebook the creation of the table `REFACTORING_MINER_bug`, to store if the refactoring produce a bug or not.

First, we import the libraries we need and, then, we read the corresponding csv files.

In [1]:
import pandas as pd
import numpy as np

In [2]:
refactoringMiner = pd.read_csv("../../../data/interim/DataPreparation/CleanData/REFACTORING_MINER_clean.csv")[['projectID', 'commitHash', 'refactoringType']]
print(refactoringMiner.shape)
refactoringMiner.head()

(57528, 3)


,projectID,commitHash,refactoringType
0,accumulo,4093a3015d6b789888077e317e535df4c8102e5d,Extract Method
1,accumulo,123bd993cff822e02242197a24f47ee36bfa3744,Extract Variable
2,accumulo,8c04c6ae5e5ba1432e40684428338ce68431766b,Extract Variable
3,accumulo,812f18b4534ae1eec41845a70a53adb783e77d61,Rename Variable
4,accumulo,812f18b4534ae1eec41845a70a53adb783e77d61,Rename Variable


In [3]:
szzFaultInducingCommits = pd.read_csv("../../../data/interim/DataPreparation/CleanData/SZZ_FAULT_INDUCING_COMMITS_clean.csv")[['projectID', 'faultFixingCommitHash', 'faultInducingCommitHash']]
print(szzFaultInducingCommits.shape)
szzFaultInducingCommits.head()

(27340, 3)


,projectID,faultFixingCommitHash,faultInducingCommitHash
0,commons-daemon,871c58c9cbfff95a4e3149e9df527401d17381fd,49cbb142a2b5d7d89aab077dc63f7646828c9408
1,commons-daemon,871c58c9cbfff95a4e3149e9df527401d17381fd,43d485a0e99c613daffba7d5b410e8e4a302e4f7
2,commons-daemon,871c58c9cbfff95a4e3149e9df527401d17381fd,4f232e47002359f1a8156b5f03b227c7bae9874a
3,commons-daemon,871c58c9cbfff95a4e3149e9df527401d17381fd,4f232e47002359f1a8156b5f03b227c7bae9874a
4,beam,e2583f5e73de50f8af128ecaa331a2e1046d2b08,c880c8ebad0e83a068301d0bbf8c76be36a59962


We merge the tables and, if doing so, there is a NaN, this indicates that it has not induced a bug.

In [4]:
induced_bug = pd.merge(refactoringMiner, szzFaultInducingCommits, how='left', left_on='commitHash', right_on='faultInducingCommitHash').drop_duplicates().reset_index()
induced_bug

,index,projectID_x,commitHash,refactoringType,projectID_y,faultFixingCommitHash,faultInducingCommitHash
0,0,accumulo,4093a3015d6b789888077e317e535df4c8102e5d,Extract Method,NaN,NaN,NaN
1,1,accumulo,123bd993cff822e02242197a24f47ee36bfa3744,Extract Variable,NaN,NaN,NaN
2,2,accumulo,8c04c6ae5e5ba1432e40684428338ce68431766b,Extract Variable,NaN,NaN,NaN
3,3,accumulo,812f18b4534ae1eec41845a70a53adb783e77d61,Rename Variable,NaN,NaN,NaN
4,8,accumulo,eac6c062b586196d32b7770d7052148acaf3c276,Extract Method,NaN,NaN,NaN
...,...,...,...,...,...,...,...
31987,189586,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Extract Method,NaN,NaN,NaN
31988,189590,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Move Attribute,NaN,NaN,NaN
31989,189592,zookeeper,344a30792bb30430a5949fa20ae69872c42394e0,Extract Variable,NaN,NaN,NaN
31990,189594,zookeeper,a1c481ceca909e32ec49ff9738b5355eb1c367a7,Move And Rename Class,NaN,NaN,NaN


In [5]:
def produce_bug(x):
  if pd.isna(x.faultFixingCommitHash):
    return False
  return True

In [6]:
induced_bug['bug'] = induced_bug.apply(lambda x: produce_bug(x), axis=1)
induced_bug

,index,projectID_x,commitHash,refactoringType,projectID_y,faultFixingCommitHash,faultInducingCommitHash,bug
0,0,accumulo,4093a3015d6b789888077e317e535df4c8102e5d,Extract Method,NaN,NaN,NaN,False
1,1,accumulo,123bd993cff822e02242197a24f47ee36bfa3744,Extract Variable,NaN,NaN,NaN,False
2,2,accumulo,8c04c6ae5e5ba1432e40684428338ce68431766b,Extract Variable,NaN,NaN,NaN,False
3,3,accumulo,812f18b4534ae1eec41845a70a53adb783e77d61,Rename Variable,NaN,NaN,NaN,False
4,8,accumulo,eac6c062b586196d32b7770d7052148acaf3c276,Extract Method,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...
31987,189586,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Extract Method,NaN,NaN,NaN,False
31988,189590,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Move Attribute,NaN,NaN,NaN,False
31989,189592,zookeeper,344a30792bb30430a5949fa20ae69872c42394e0,Extract Variable,NaN,NaN,NaN,False
31990,189594,zookeeper,a1c481ceca909e32ec49ff9738b5355eb1c367a7,Move And Rename Class,NaN,NaN,NaN,False


In [7]:
induced_bug.bug.value_counts()

False    18497
True     13495
Name: bug, dtype: int64

In [8]:
induced_bug = induced_bug[['projectID_x', 'commitHash', 'refactoringType', 'bug']].rename(columns={'projectID_x': 'projectID'})
induced_bug

,projectID,commitHash,refactoringType,bug
0,accumulo,4093a3015d6b789888077e317e535df4c8102e5d,Extract Method,False
1,accumulo,123bd993cff822e02242197a24f47ee36bfa3744,Extract Variable,False
2,accumulo,8c04c6ae5e5ba1432e40684428338ce68431766b,Extract Variable,False
3,accumulo,812f18b4534ae1eec41845a70a53adb783e77d61,Rename Variable,False
4,accumulo,eac6c062b586196d32b7770d7052148acaf3c276,Extract Method,False
...,...,...,...,...
31987,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Extract Method,False
31988,zookeeper,2d5dd1da4d144cd1ab76edda05e4faa2d6f368e3,Move Attribute,False
31989,zookeeper,344a30792bb30430a5949fa20ae69872c42394e0,Extract Variable,False
31990,zookeeper,a1c481ceca909e32ec49ff9738b5355eb1c367a7,Move And Rename Class,False


We save this table in a new csv.

In [9]:
induced_bug.to_csv('../../../data/interim/DataPreparation/ConstructData/REFACTORING_MINER_bug.csv', header=True)